In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,SentenceTransformersTokenTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import chromadb 
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import os
import shutil
from langchain_community.document_loaders import Docx2txtLoader

from pypdf import PdfReader

import os
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
openai_client = OpenAI()


In [2]:

import umap
import numpy as np
from tqdm import tqdm

/home/eyaya/Desktop/Challenges/Week_11/Contract_Advisor_RAG/venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

import sys
sys.path.append('../')
from src.utils import word_wrap

In [46]:
vectordb_path = '../data/chromadb/'
data_path = '../data/contract_data/'

In [47]:
reader = PdfReader(data_path+'Raptor.pdf')

In [48]:
pdf_texts = [p.extract_text().strip() for p in reader.pages]
pdf_texts = [text for text in pdf_texts if text]

In [49]:
print(word_wrap(pdf_texts[0]))

[R&G Draft 12.__.2021] STOCK PURCHASE AGREEMENT BY AND AMONG [BUYER],
[TARGET COMP ANY], THE SELLERS LISTED ON SCHEDULE I HERET O AND THE
SELLERS’ REPRESENT ATIVE NAMED HEREIN Dated as of [●] [This document
is intended solely to facilitate discussions among the parties
identified herein.  Neither this document nor such discussions are
intended to create, nor will either or both be  deemed to create, a
legally binding or enforceable offer or agreement of any type or
nature,  unless and until a definitive written agreement is executed
and delivered by each of the parties  hereto. This document shall be
kept confidential pursuant to the terms of the Confidentiality
Agreement entered into by the parties and, if applicable, its
affiliates with respect to the subject  matter hereof.] 112923184_5


In [50]:
character_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n","\n",".", " ",""],
        chunk_size=500,
        chunk_overlap=10,
    )
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

In [51]:
print('Total chunk',len(character_split_texts))

Total chunk 490


In [52]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

In [53]:
token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

stock purchase agreement this stock purchase agreement ( this “
agreement ” ) is made and entered into as of [ ● ] by and among [
parent ], a [ ● ] corporation ( “ parent ” ) ], [ buyer ], a delaware
corporation and a wholly owned subsidiary of parent ( “ buyer ” ) 1, [
target comp any ], a delaware corporation ( the “ company ” ), each of
the holders of outstanding shares of capital stock of the company
listed on schedule i hereto ( respectively, the “ shareholders ” or
the “ sellers ” ) 2, and [ ● ], in

Total chunks: 490


In [54]:
embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[[-0.010687097907066345, -0.02267240546643734, 0.058891016989946365, -0.026844708248972893, -0.016043977811932564, 0.007356878370046616, 0.03913184255361557, 0.02374662272632122, 0.04208064079284668, -0.04585960507392883, 0.07091499865055084, -0.020057646557688713, 0.007743957918137312, -0.020984595641493797, 0.061448659747838974, 0.007069782819598913, 0.00023805622186046094, 0.025507736951112747, 0.04417411983013153, -0.008094425313174725, -0.08181234449148178, -0.018982628360390663, 0.001050553168170154, 0.031059488654136658, 0.026162974536418915, 0.007653172127902508, 0.020977508276700974, 0.033604495227336884, 0.017255356535315514, -0.03667173162102699, -0.033191192895174026, -0.017380716279149055, 0.08637655526399612, 0.04638750106096268, -0.008988943882286549, 0.03413565829396248, 0.01322031207382679, -0.06021418794989586, 0.01641496829688549, -0.001064987969584763, 0.007121656555682421, 0.024552837014198303, 0.014552944339811802, -0.026268137618899345, 0.006666858214884996, 0.03

In [55]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection(name='contract', embedding_function=embedding_function)

UniqueConstraintError: Collection contract already exists

In [56]:
ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

490

In [61]:
query = "Is escrow amount greate than the Retention Amount ?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

gains of such person ( including stock appreciation, phantom stock,
profit participation or other similar rights ). “ escrow amount ”
means, $ 1, 000, 000 “ facilities ” means any buildings, plants,
improvements or structures located on the real property. “ family
member ” means, with respect to any individual, ( a ) such person ’ s
spouse, ( b ) each parent, brother, sister or child of such person or
such person ’ s spouse, ( c ) the spouse of any person described in
clause ( b ) above, ( d ) each child


in accordance with the escrow agreement. the escrow amount shall be
held and, subject to section 2. 07, released to the company
securityholders in accordance with the provisions of the escrow
agreement with the company securityholders being entitled to share in
such released amounts in accordance with their pro rata percentages.
from and after the closing, buyer and the sellers ’ representative
will direct the escrow agent to disburse payments from the escrow
account in accordance wi

In [62]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [63]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print(word_wrap(output))

Based on the information provided from the annual report, the escrow
amount is $1,000,000. However, there is no specific mention or
comparison to a "Retention Amount" in the information provided.
Therefore, it is not possible to determine if the escrow amount is
greater than the Retention Amount based on the information provided.


In [21]:
embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)

/home/eyaya/Desktop/Challenges/Week_11/Contract_Advisor_RAG/venv2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [22]:
def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings),2))
    for i, embedding in enumerate(tqdm(embeddings)): 
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings

In [23]:
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

100%|██████████| 490/490 [04:50<00:00,  1.69it/s]


In [25]:
import plotly.express as px
from umap import UMAP

# Load the Iris dataset


# Create a UMAP model and fit it to the data
umap_2d = UMAP(n_components=2, random_state=0)


# Visualize the UMAP projections
fig = px.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1])
fig.show()


In [26]:
query = "How much is the escrow amount?"

results = chroma_collection.query(query_texts=query, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

gains of such person ( including stock appreciation, phantom stock,
profit participation or other similar rights ). “ escrow amount ”
means, $ 1, 000, 000 “ facilities ” means any buildings, plants,
improvements or structures located on the real property. “ family
member ” means, with respect to any individual, ( a ) such person ’ s
spouse, ( b ) each parent, brother, sister or child of such person or
such person ’ s spouse, ( c ) the spouse of any person described in
clause ( b ) above, ( d ) each child

in accordance with the escrow agreement. the escrow amount shall be
held and, subject to section 2. 07, released to the company
securityholders in accordance with the provisions of the escrow
agreement with the company securityholders being entitled to share in
such released amounts in accordance with their pro rata percentages.
from and after the closing, buyer and the sellers ’ representative
will direct the escrow agent to disburse payments from the escrow
account in accordance wit

In [27]:
query_embedding = embedding_function([query])[0]
retrieved_embeddings = results['embeddings'][0]

projected_query_embedding = project_embeddings([query_embedding], umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

100%|██████████| 5/5 [00:02<00:00,  1.71it/s]


In [36]:
# Plot the projected query and retrieved documents in the embedding space
import plotly.graph_objects as go
trace1 = go.Scatter(x=projected_dataset_embeddings[:, 0], y=projected_dataset_embeddings[:, 1],mode="markers")
trace2 = go.Scatter(x=projected_query_embedding[:, 0], y=projected_query_embedding[:, 1],mode="markers")
trace3 = go.Scatter(x=projected_retrieved_embeddings[:, 0], y=projected_retrieved_embeddings[:, 1],mode="markers")

# Create a figure and add the traces
fig = go.Figure(data=[trace1, trace2,trace3])
fig.show()

In [40]:
from langchain.text_splitter import NLTKTextSplitter, SpacyTextSplitter


In [41]:
spliter = NLTKTextSplitter()

In [44]:
character_split_texts = spliter.split_text(pdf_texts[0])

In [45]:
character_split_texts

['[R&G\nDraft\n12.__.2021]\nSTOCK\nPURCHASE\nAGREEMENT\nBY\nAND\nAMONG\n[BUYER],\n[TARGET\nCOMP ANY],\nTHE\nSELLERS\nLISTED\nON\nSCHEDULE\nI\nHERET O\nAND\nTHE\nSELLERS’\nREPRESENT ATIVE\nNAMED\nHEREIN\nDated\nas\nof\n[●]\n[This\ndocument\nis\nintended\nsolely\nto\nfacilitate\ndiscussions\namong\nthe\nparties\nidentified\nherein.\n\nNeither\nthis\ndocument\nnor\nsuch\ndiscussions\nare\nintended\nto\ncreate,\nnor\nwill\neither\nor\nboth\nbe \ndeemed\nto\ncreate,\na\nlegally\nbinding\nor\nenforceable\noffer\nor\nagreement\nof\nany\ntype\nor\nnature, \nunless\nand\nuntil\na\ndefinitive\nwritten\nagreement\nis\nexecuted\nand\ndelivered\nby\neach\nof\nthe\nparties \nhereto.\n\nThis\ndocument\nshall\nbe\nkept\nconfidential\npursuant\nto\nthe\nterms\nof\nthe\nConfidentiality \nAgreement\nentered\ninto\nby\nthe\nparties\nand,\nif\napplicable,\nits\naffiliates\nwith\nrespect\nto\nthe\nsubject \nmatter\nhereof.]\n\n112923184_5']